<a href="https://colab.research.google.com/github/Saumya0330/Saumya-USC420/blob/main/cog_assignment12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lab Assignemnt-12
Cognitive Computing UCS420


Deploy Chatbot for Healthcare Applications

In [2]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import Dataset

data = {
    "input_text": [
        "Hi", "Hello", "I have a headache", "What should I do if I have a fever?", "I feel dizzy",
        "What should I eat for a cold?", "How to stay healthy?", "What should I do in case of a cut?",
        "How much water should I drink daily?", "Thank you", "Bye"
    ],
    "output_text": [
        "Hello! I am HealthBot. How can I assist you today?", "Hi! Do you have a health-related question?",
        "You should rest, stay hydrated, and take a mild pain reliever if needed.",
        "Drink plenty of fluids and rest. If the fever persists, please consult a doctor.",
        "Sit down, breathe deeply, and drink water. If it continues, seek medical help.",
        "Warm fluids, soups, citrus fruits, and light meals help during a cold.",
        "Eat balanced meals, exercise regularly, stay hydrated, and get enough sleep.",
        "Clean the wound with water, apply antiseptic, and cover it with a clean bandage.",
        "Generally, 2 to 3 liters per day is recommended, but it varies based on your activity.",
        "You’re welcome! Take care.", "Goodbye! Stay healthy."
    ]
}

dataset = Dataset.from_dict(data)


In [8]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def preprocess_data(examples):
    inputs = [ex for ex in examples['input_text']]
    outputs = [ex for ex in examples['output_text']]
    input_ids = tokenizer(inputs, truncation=True, padding="max_length", max_length=64, return_tensors="pt").input_ids
    output_ids = tokenizer(outputs, truncation=True, padding="max_length", max_length=64, return_tensors="pt").input_ids
    return {"input_ids": input_ids, "labels": output_ids}

dataset = dataset.map(preprocess_data, batched=True, batch_size=1) #added batch_size
#The following line was causing an error as tokenizer.pad_token == tokenizer.eos_token is a boolean value and not callable
#dataset = dataset.map(preprocess_data, batched=True, tokenizer.pad_token == tokenizer.eos_token)

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [10]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="no",
    weight_decay=0.01
)

# Set up the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-1060588f015d>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sgupta6_be23 (sgupta6_be23-thapar-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=9, training_loss=3.9193602667914496, metrics={'train_runtime': 164.574, 'train_samples_per_second': 0.201, 'train_steps_per_second': 0.055, 'total_flos': 1077829632000.0, 'train_loss': 3.9193602667914496, 'epoch': 3.0})

In [11]:
model.save_pretrained("healthbot_model")
tokenizer.save_pretrained("healthbot_model")


('healthbot_model/tokenizer_config.json',
 'healthbot_model/special_tokens_map.json',
 'healthbot_model/vocab.json',
 'healthbot_model/merges.txt',
 'healthbot_model/added_tokens.json')

In [34]:
# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("healthbot_model")
tokenizer = GPT2Tokenizer.from_pretrained("healthbot_model")

# Test the chatbot with an example input
input_text = "how to fix fever"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate the response
output = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2)

# Decode and print the response
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


how to fix fever, and to cure it.

The first thing to do is to get the fever to stop. If you have a fever and you are not able to go to the doctor, you can get a cold. You can go
